<a href="https://colab.research.google.com/github/huavanson/CS114.K21/blob/master/sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mô tả bài toán : Phân biệt 1 tiêu đề có tính châm biếm hay không 


*   input : 1 title 
*   output : is sarcasm or not sarcasm 








Dataset : lấy từ kaggle (link dataset:https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection)
Mô tả dataset : có 3 columns

*   headline : là tiêu đề của 1 bài viết 
*   is_sarcastic: 0 là not sarcasm , 1 là is sarcasm


*   article_link: link bài viết 





Dataset lấy trên kaggle có 2 file nhưng em chỉ sử dụng 1 file 

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sonhuavan","key":"899cfa81c0f3f4051aee96eb7e74b10f"}'}

Sau khi up file kaggle.json lên thì tạo 1 thư mục có tên .kaggle đưa file vừa up vào 

In [2]:
!ls -lha kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 65 Jun 24 14:15 kaggle.json


Bước này Download dataset trên kaggle về 


In [3]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 54.6MB/s]


Check xem các thư viện cần thiết đã được install vào chưa 


In [ ]:
!pip install numpy
!pip install pandas 
!pip install sklearn

In [4]:
import json
import pandas as pd, numpy as np, re, time
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [5]:
!unzip news-headlines-dataset-for-sarcasm-detection.zip

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


Đọc dataset bằng Pandas và check số lượng data 

In [138]:
df=pd.read_json('Sarcasm_Headlines_Dataset.json',lines=True)
# shape and head
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [7]:
df.shape

(26709, 3)

check xem data có bị missing value hay không

In [8]:
print(df.isnull().any(axis = 0))

article_link    False
headline        False
is_sarcastic    False
dtype: bool


Column article_link không cần thiết để detect 1 headline có sarcasm hay không nên có thể được lượt bỏ 


In [139]:
df = df.drop(['article_link'], axis=1)

Dữ liệu không bị missing value nhưng cột tiêu đề nên được loại bỏ các kí tự không cần thiết vì vậy em sử dụng Regular Expression ( Biểu thức chính quy) để loại bỏ các kí tự đặc biệt đó ví dụ dấu nháy dấu chấm v.v

In [140]:
df['headline'] = df['headline'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))


In [11]:
df.head(5)

,headline,is_sarcastic
0,former versace store clerk sues over secret b...,0
1,the roseanne revival catches up to our thorn...,0
2,mom starting to fear son s web series closest ...,1
3,boehner just wants wife to listen not come up...,1
4,j k rowling wishes snape happy birthday in th...,0


In [141]:
features = df['headline']
labels = df['is_sarcastic']
features_compare=features


Xử dụng PorterStemmer để cắt giảm 1 từ thành từ gốc của nó 
Ví dụ : "reading" or "reader"
Sau khi stemming sẽ trở thành "read" ( cho cả 2 từ trên)

In [142]:
ps = PorterStemmer()
features = features.apply(lambda x: x.split())
features = features.apply(lambda x : ' '.join([ps.stem(word) for word in x]))

So sánh chuỗi trước và sau khi stemming 

In [14]:
print(features_compare[0])
print(features[0])

former versace store clerk sues over secret  black code  for minority shoppers
former versac store clerk sue over secret black code for minor shopper


tf-idf ( term frequency–inverse document frequency) hay còn gọi là tần số xuất hiện- tần số nghịch đảo của 1 từ trong 1 văn bản . Kỹ thuật này sử dụng để đếm số lần xuất hiện của 1 từ trong 1 văn bản nhiều hoặc ít để tìm ra những từ quan trọng trong 1 văn bản 

Link tham khảo tf-idf :https://en.wikipedia.org/wiki/Tf%E2%80%93idf

TfidfVectorizer dùng để chuyển đổi bộ dữ liệu sang 1 ma trận có các thông số tf-idf dưới đây là link thư viện scikitlearn


https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

TfidfVectorizer train cho model học để tìm ra số lượng từ khác nhau trong bộ dataset . ví dụ sau khi đặt max_features=5000
*   bộ dataset gồm 1 dòng headline có 2 từ : Civil Whites thì sau khi được học sẽ trả về 1 ma trận (1,2)  
*  bộ dataset gồm 20000 dòng headline : thì sau khi học được sẽ trả về ma trận (20000,5000) 



Vậy max_features có nghĩa là quy định số lượng từ khác nhau tối đa mà model có thể học để tìm ra trong 1 bộ dataset 

In [143]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 5000)
features = list(features)
features = tv.fit_transform(features).toarray()

features sau khi được chuyển sang ma trận 


In [16]:
features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
print(features.shape)

(26709, 5000)


Vì dữ liệu khá nhiều nên em sẽ chia theo tỷ lệ 90-10

In [18]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.1, random_state = 0)

Dùng thử 4 model để train


---



---



In [178]:
from sklearn.metrics import accuracy_score
# model 1:-
# linear support vector classifier
lsvc = LinearSVC()
# training model
lsvc.fit(features_train, labels_train)
# accuracy tập train và tập test
print('lsvc')
print(lsvc.score(features_train, labels_train))
print(lsvc.score(features_test, labels_test))   


# model 2:-
#  Gaussuan Naive Bayes
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
print('gnb')
print(gnb.score(features_train, labels_train))
print(gnb.score(features_test, labels_test))  

# model 3:-
# Logistic Regression
lr = LogisticRegression()
lr.fit(features_train, labels_train)
print('lr')
print(lr.score(features_train, labels_train)) 
print(lr.score(features_test, labels_test))     

# model 4:
# Random Forest Classifier
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print('rfc')
print(rfc.score(features_train, labels_train))
print(rfc.score(features_test, labels_test))    



lsvc
0.9110574923038522
0.8360164732309996
gnb
0.7826358266078709
0.7162111568700861
lr
0.8824777435726766
0.8375140396855111
rfc
0.9895998003161661
0.7963309621864471


Crawl Data trên trang https://www.theonion.com/latest để lấy những head line sarcasm

Install 1 số thư viện cần thiết

In [ ]:
!pip install beautifulsoup4
! pip install requests

Hàm Crawl data sử dụng beautifulSoup 

BeautifulSoup(BS) là 1 package của python dùng để cạo thông tin từ 1 trang web . BS cho phép phân tích nội dung HTML của 1 URL nhất định và truy cập các phần tử của nó bằng cách xác định các thẻ và các thuộc tính . 

Link tham khảo cách cào dữ liệu trên 1 trang web :https://towardsdatascience.com/web-scraping-news-articles-in-python-9dd605799558

In [49]:
import requests
from bs4 import BeautifulSoup
def crawl(url, data):
  r1 = requests.get(url)
  coverpage = r1.content
  soup1 = BeautifulSoup(coverpage, 'html5lib')
  coverpage_news = soup1.find_all("h2", class_="sc-759qgu-0 cYlVdn cw4lnv-6 eXwNRE")
  for i in coverpage_news:
     data.append(i.get_text())

để cung cấp BeautifulSoup mã HTML của bất kỳ trang nào,  sẽ cần import 1 cái module requests

Vì em cần cạo khoảng 1000 headline nhưng mỗi trang trên trang web :"https://www.theonion.com/latest" chỉ hiển thị khoảng 20 headline, trang kế tiếp có URL khác , để chuyển sang trang tiếp theo em trích xuất vào chổ chuyển trang cụ thể là nút(more stories trong trang web) tìm ra đuôi href gắn vào link đầu tiên để truy cập vào trang kế cào tiếp 20 headline, như vậy dữ liệu cào xuống sẽ không trùng nhau 




In [ ]:
url = "https://www.theonion.com/latest"
more_stories = ''
data= []
while len(data)<1000:
  crawl(url, data)
  r1 = requests.get(url)
  coverpage = r1.content
  soup1 = BeautifulSoup(coverpage, 'html5lib')
  more_stories = soup1.find_all("a", class_="sc-1out364-0 hMndXN js_link")
  more_stories_next_link = more_stories[-1].get('href')
  url="https://www.theonion.com/latest"+more_stories_next_link
  print(url)

Độ dài data sau khi cào

In [57]:
data=list(dict.fromkeys(data))
print(len(data))

1000


Hàm crawl được sửa lại 1 chút để cào ở trang web khác . "https://www.avclub.com/latest là trang web chuyên tin chính thống (em không cào được trên trang huffpost vì gọi request nó không trả lại gì hết ạ )

In [149]:
def crawl1(url, data1):
  r1 = requests.get(url)
  coverpage = r1.content
  soup1 = BeautifulSoup(coverpage, 'html5lib')
  coverpage_news = soup1.find_all("h2", class_='sc-759qgu-0 sc-759qgu-1 kYoTly sc-3kpz0l-8 PRCbV')
  for i in coverpage_news:
    data1.append(i.get_text())

Cách cào tương tự với trang web ở trên 

In [151]:
url = "https://www.avclub.com/latest"
data1= []
stories=''
while len(data1)<1000:
  crawl1(url+stories, data1)
  r1 = requests.get(url+stories)
  coverpage = r1.content
  soup1 = BeautifulSoup(coverpage, 'html5lib')
  t = soup1.find_all("a", class_="sc-1out364-0 hMndXN js_link" )
  stories=t[-1].get('href')

Độ dài sau khi cào ở web "https://www.avclub.com/latest 

In [167]:
data1=list(dict.fromkeys(data1))
len(data1)

1008

Gán label cho từng cái headline 

In [158]:
headline = data[:1000] + data1[:1000] 
is_sarcastic=[]
for i in range(0,1000):
  is_sarcastic.append(1)  
for i in range(0,1000):
  is_sarcastic.append(0)  


Biểu diễn data dưới dạng bảng 

In [159]:
data_test = pd.DataFrame({
    'headline': headline,
    'is_sarcasm': is_sarcastic
})
data_test

,headline,is_sarcasm
0,IBM Condemns Use Of Facial Recognition Softwar...,1
1,Cat Sticks Paw Around Corner Like Life Some Bi...,1
2,Milky Way Could Harbor 6 Billion Earth-Like Pl...,1
3,Realtor Thinks Flourishing Neighborhood Full O...,1
4,Civil Whites,1
...,...,...
1995,"Ishtar, the ultimate Hollywood punchline, is a...",0
1996,The CW to kill some time in quarantine by airi...,0
1997,Author Caitlin Moran picks her top 5 coming-of...,0
1998,Watch Rivers Cuomo cover Nirvana’s “Heart Shap...,0


Sử dụng hàm shuffle để đảo lộn vị trí data 

In [160]:
from sklearn.utils import shuffle
df = shuffle(data_test)
df

,headline,is_sarcasm
969,Timberwolves Fire Clerk On Accounts Receivable...,1
1349,You Don’t Nomi spotlights the cult of Showgirl...,0
1429,Enjoy some on-point parodies of Tom Cotton's t...,0
2,Milky Way Could Harbor 6 Billion Earth-Like Pl...,1
1967,"Despite its sunny hero, Stargirl goes to some ...",0
...,...,...
1100,Love Is Blind showrunner Sam Dean and HBO Max ...,0
1527,"K-pop fans do their part, defeat Dallas police...",0
958,Obama Kind Of Hurt No One’s Even Asked For His...,1
1427,"Ari Aster says his next movie, a ""zonky nightm...",0


Xử lí data trước khi predict 

In [161]:
df['headline'] = df['headline'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))
x_valid = df['headline']
y_valid = df['is_sarcasm']
ps = PorterStemmer()
x_valid = x_valid.apply(lambda x: x.split())
x_valid = x_valid.apply(lambda x : ' '.join([ps.stem(word) for word in x]))
x_valid = list(x_valid)
x_valid = tv.transform(x_valid).toarray()

Tính độ accuracy với 4 model được train ở trên 

In [170]:
from sklearn.metrics import accuracy_score
pred1=lsvc.predict(x_valid)
pred2=gnb.predict(x_valid)
pred3=lr.predict(x_valid)
pred4=rfc.predict(x_valid)
acc1=accuracy_score(pred1,y_valid)
acc2=accuracy_score(pred2,y_valid)
acc3=accuracy_score(pred3,y_valid)
acc4=accuracy_score(pred4,y_valid)
print('lsvc :',acc1)
print('gnb :',acc2)
print('lr :',acc3)
print('rfc :',acc4)

lsvc : 0.692
gnb : 0.6275
lr : 0.691
rfc : 0.6785


Performance của model trên dataset đã cho tốt hơn so với 2000 headline mới dựa vào độ accuracy_score của từng model lsvc,gnb,lr,rfc(0.836-0.692,0.716-0.6275,0.837-0.691,0.796-0.6785)

Thử predict 1 headline mới hoàn toàn (thực ra là em lấy dòng thứ 4 của data em crawl được từ trang :"https://www.theonion.com/latest"

Em đưa cái headline này trở thành 1 dataset có 1 dòng sau đó áp dụng 3 kỹ thuật xử lí ở trên(re,stemming,tfidf) và đưa vào 1 model train sẵn predict thử 

In [175]:
new_input=['Civil White']
data_input = pd.DataFrame({
    'headline': new_input,
})

data_input['headline'] =data_input['headline'] .apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))
ps = PorterStemmer()
data_input['headline'] = data_input['headline'].apply(lambda x: x.split())
data_input['headline']= data_input['headline'].apply(lambda x : ' '.join([ps.stem(word) for word in x]))
arr_input = tv.transform(data_input['headline']).toarray()

Kết quả trả về là 1 có nghĩa là headline này là sarcasm

In [176]:
lsvc.predict(arr_input)

array([1])

Nhận xét về bài toán : Bài toán này để có được độ accuracy cao đầu tiên phải có nhiều data từ đó model có thể học được nhiều từ quan trọng trong các headline sarcasm ,thứ 2 là phải chọn các kỹ thuật xử lí data phù hợp và cuối cùng là ngoài nhiều ra thì data còn phải đa dạng từ ngữ về nhiều lĩnh vực ...